## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-12-04-20-41 +0000,bbc,"Gaza ceasefire talks on verge of collapse, Pal...",https://www.bbc.com/news/articles/cqjq9p87vdvo
1,2025-07-12-04-01-07 +0000,nyt,How to Panic Italians? Jack Up the Price of Es...,https://www.nytimes.com/2025/07/12/world/europ...
2,2025-07-12-03-00-16 +0000,bbc,Federal judge says voice-over artists' AI laws...,https://www.bbc.com/news/articles/cedgzj8z1wjo
3,2025-07-12-02-59-22 +0000,bbc,Why cockpit audio deepens the mystery of Air I...,https://www.bbc.com/news/articles/cx2gy78gpnqo
4,2025-07-12-01-58-41 +0000,latimes,Man sets California probation officer on fire ...,https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2351/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
127,trump,13
51,immigration,6
153,canada,5
297,will,5
27,california,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
60,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,35
37,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...,31
29,2025-07-11-21-50-58 +0000,nyt,"As Trump Sows Tariff Confusion, Rules of Globa...",https://www.nytimes.com/2025/07/11/world/europ...,29
40,2025-07-11-19-04-37 +0000,nyt,Trump Says NATO Countries Will Buy Weapons to ...,https://www.nytimes.com/2025/07/11/world/europ...,29
20,2025-07-11-23-05-58 +0000,bbc,Canada's Carney talked tough on Trump - now so...,https://www.bbc.com/news/articles/cvg81518yyyo,25


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
60,35,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
9,23,2025-07-12-01-09-58 +0000,latimes,California mayor warns residents amid immigrat...,https://www.latimes.com/california/story/2025-...
5,20,2025-07-12-01-53-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...
50,19,2025-07-11-16-57-34 +0000,bbc,Mother mourns 'beautiful' 12-year-old shot whi...,https://www.bbc.com/news/articles/cd6gzx85gdqo
81,19,2025-07-11-10-00-00 +0000,latimes,"Arellano: 'La migra, la migra': Inside Hunting...",https://www.latimes.com/california/story/2025-...
37,18,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...
47,17,2025-07-11-17-37-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...
69,15,2025-07-11-13-09-38 +0000,nyt,Kurdish PKK Fighters Burn Weapons in Step Towa...,https://www.nytimes.com/2025/07/11/world/middl...
16,14,2025-07-12-00-16-13 +0000,latimes,SoCal surgery center staff confront ICE agents...,https://www.latimes.com/california/story/2025-...
78,14,2025-07-11-10-00-00 +0000,latimes,How a Supreme Court win for public health bols...,https://www.latimes.com/politics/story/2025-07...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
